In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import openai
import io
import pandas as pd
import tqdm
import sys
from datetime import datetime


sys.path.append('../hot_links')
# TODO Something like this in dataset.py
# from hot_links.config import PROCESSED_DATA_DIR, RAW_DATA_DIR

from hot_links.vector_db_utils import *


2025-08-28 06:54:18.433 | INFO     | hot_links.config:<module>:11 - PROJ_ROOT path is: /workspaces/hot-links


## OpenAI Vector Store

Based on [OpenAI documentation](https://platform.openai.com/docs/guides/tools-file-search#how-to-use)

In [4]:
now = datetime.now()
unix_time = now.timestamp()
# Make a time attribute


In [5]:
unix_time 

1756285588.642007

In [6]:
vector_store_id = create_vs("test_15")
file_id = push_file_to_cloud("test_15", "The key to the universe is 24")
res = add_file_to_db(file_id, vector_store_id, attributes={'time':unix_time})

In [7]:
response = client.responses.create(
    model="gpt-4.1",
    input="I need the key to the universe... What might it be? Please check in the vector store.",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
        "filters": {
            "type": "lt",
            "key": "time",
            "value": datetime.now().timestamp()
        }
    }]
)
print(response)
# Then query the store with filters

Response(id='resp_68aebf55f1d881908ae21ab8523f05de06121715ae4800a9', created_at=1756282710.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseFileSearchToolCall(id='fs_68aebf56eff88190b6e9234825c1b68206121715ae4800a9', queries=['key to the universe'], status='completed', type='file_search_call', results=None), ResponseOutputMessage(id='msg_68aebf5801e48190be3e51b48fcfa36306121715ae4800a9', content=[ResponseOutputText(annotations=[AnnotationFileCitation(file_id='file-YAwezHhAVEmax1MrFbFr7E', filename='test_12.txt', index=29, type='file_citation')], text='The key to the universe is 24.', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FileSearchTool(type='file_search', vector_store_ids=['vs_68aebf4484b48191981bd5f0c89a35ca'], filters=ComparisonFilter(key='time', type='lt', value=1756282708.7698

In [9]:
#response.error
response.output[1].content[0].text

'The key to the universe is 24.'

In [7]:
for vs in get_all_vs():
    print(vs.id,vs.name)

vs_68aecaa3def88191804f03510e690fb5 test_15
vs_68aec5911d54819189b3e643a653d59d test_14
vs_68aec56252388191924e885daeceed79 test_13
vs_68aebf4484b48191981bd5f0c89a35ca test_12
vs_68aebb80bbb08191ac68f228a08a83d4 test_11
vs_68aebae352f48191927c4b1260fb7f73 test_10
vs_68aeba2add0c81919da4e4d10fc47630 test_9
vs_68aeb9afea2081919f8be4f1ee611a82 test_8
vs_68aeb89b6c088191b75c037f1405d714 test_7
vs_68aeb734ba888191830f01f1ec997b47 test_6
vs_68aeb6ef7fb88191b1698338b66a6eec test_5
vs_68aeb6bfb31c81919e722bbd2f50c7b5 test_4
vs_68acb4af008c81919b4924472e209446 hot_links
vs_68acae441e188191a605f82b03a98305 hot_links
vs_68ac553757d88191873c6900f9c90c0c hot_links_final2
vs_68ac54fb91b0819180b65563cb88062e hot_links_final2
vs_68ac547571e881918c5b003281473561 hot_links_final
vs_68ac4cff5f1c81919b3bfda45ea803bb hot_links
vs_68ab6e552c348191b066ae9baa26b73f hot_links
vs_68ab680fb1908191b4c9cf36e59ab145 hot_links
vs_68ab595999388191ad48c47f6d09bae2 hot_links


In [8]:
delete_all_files()

16it [00:07,  2.11it/s]


## Try adding real files

In [3]:
df = pd.read_csv('../data/interim/snowballed.csv')

In [4]:
vector_store_id = None

for vs in get_all_vs():
    print(vs.id,vs.name)
    
    if vs.name == 'hot_links':
        vector_store_id = vs.id
    
    # client.vector_stores.delete(vs.id)

vs_68aee23606108191acbb1b3f0dfc4f69 hot_links


In [ ]:
vector_store_id = get_vector_store_id()

In [ ]:
if not vector_store_id:
    vector_store_id = create_vs("hot_links")

In [6]:
all_files = set(map(lambda x:x.filename, get_all_files()))
# Get all the files already in the vector store to skip omes already there

n_added = 0
n_skipped = 0

for row in tqdm.tqdm(df.head(13000).iterrows()):
    date,url,content = row[1][['webPublicationDate', 'webUrl','bodyContent']]

    if not url+'.txt' in all_files:
        
        logging.info("Adding %s", url)
        n_added+=1

        file_id = push_file_to_cloud(url, content)
        res = add_file_to_db(file_id, vector_store_id, attributes={'time':convert_date_to_epoch(date)})
    else:
        logging.info("Skipping %s", url)
        n_skipped+=1

logging.info("Finished processing files. Added: %d, Skipped: %d", n_added, n_skipped)
print("Finished processing files. Added: {:d}, Skipped: {:d}".format(n_added, n_skipped))

12667it [58:47,  3.59it/s]  


BadRequestError: Error code: 400 - {'error': {'message': 'Too many files in vector store', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [ ]:
response = query_db(vector_store_id, "Which countries first imposed travel restrictions due to covid-19?", pd.to_datetime('2020-01-01').value//10**9)
print(response.output[1].content[0].text)

It seems I couldn't find specific information about the first countries to impose travel restrictions due to COVID-19 in the uploaded documents. 

However, in general historical context, countries like China were among the first to impose travel restrictions, particularly in January 2020, followed by others such as Italy and various nations in the Asia-Pacific region. If you have specific documents that might contain this information, please share more details, or I can help with other inquiries!


In [7]:
response = query_db(vector_store_id, "What are the sanctions on vladimir putin?", pd.to_datetime('2023-01-01').value//10**9)
print(response.output[1].content[0].text)

Vladimir Putin, as the President of Russia, has been directly targeted by sanctions from multiple Western countries in response to his ordering of the invasion of Ukraine. Here is a summary of the specific sanctions imposed on him:

**Direct Sanctions (Personal)**
- The EU, US, UK, Australia, Canada, Japan, and New Zealand have imposed sanctions directly targeting Vladimir Putin. These personal sanctions include:
  - Asset freezes: Any foreign assets held by Putin in the EU, US, UK, and other participating countries are frozen.
  - Travel bans: Putin is banned from traveling to these jurisdictions.
- Similar sanctions have also been applied to other top Russian officials, such as the Foreign Minister Sergei Lavrov and the Defence Minister Sergei Shoigu.

**Sanctions on Putin’s Network**
- Oligarchs and individuals considered to be in Putin’s close circle, sometimes referred to as "Putin’s wallet," have also attracted asset freezes and travel bans. These include businesspeople and offic

In [8]:
response = query_db(vector_store_id, "What are the positive effects of breastfeeding for poor mothers?", pd.to_datetime('2020-01-01').value//10**9)
print(response.output[1].content[0].text)

It appears that the uploaded files do not contain specific information directly addressing the positive effects of breastfeeding for poor mothers. However, I can provide a well-supported summary based on established public health knowledge:

### Positive Effects of Breastfeeding for Poor Mothers

1. **Economic Savings**: Breastfeeding is free compared to formula feeding, which can place a substantial financial burden on low-income families. Avoiding formula helps save money that can be used for other essential needs.

2. **Improved Maternal Health**: Breastfeeding is associated with a reduced risk of certain diseases for mothers, including breast and ovarian cancers, type 2 diabetes, and postpartum depression. These health benefits can lower long-term healthcare costs and improve a mother’s ability to care for her family.

3. **Convenience and Hygiene**: Breast milk is always available, at the right temperature, and does not require sterilization or preparation—important factors where 

In [10]:
results = search_db(vector_store_id, "Which countries first imposed travel restrictions due to covid-19?")

In [11]:
for r in results:
    print(r.score)
    print(r.filename)
    print(r.content)
    print()

0.5355754943756722
https://www.theguardian.com/australia-news/2022/jan/24/what-does-australias-removal-from-the-european-councils-travel-white-list-mean.txt
[Content(text='The European Council has removed Australia from the “white list”, a register of countries from which it recommends travel restrictions should be lifted. The council’s list of epidemiologically-safe third countries is non-binding and countries within the European Union are able to set their own border requirements, which may involve quarantine or testing requirements. What did the European Council decide? The European Council said on 17 January it had updated its list of countries for which travel restrictions should be lifted, and removed Argentina, Australia and Canada. The decision is non-binding on European Union member states and reviewed every two weeks. Under the criteria used by the European Council, all restrictions on travellers from a third country can be lifted if the rate of Covid-19 infections in the pas

In [14]:
type(r)

openai.types.vector_store_search_response.VectorStoreSearchResponse

In [70]:
snippet = """Donald Trump imposed 50% tariffs on most US imports from India, making good on a threat to punish one of the world’s largest economies over its purchases of discounted Russian oil.

The tariffs, which came into effect just after midnight on Wednesday in Washington, risk inflicting significant damage on the Indian economy and further disrupting global supply chains.

US tariffs of 25% on Indian goods went into force earlier this month, but Trump announced plans to double the rate, citing New Delhi’s purchases of Russian oil, which the White House has argued is indirectly funding Russia’s war against Ukraine.

The US president has significantly increased US tariffs on goods from much of the world since reentering the White House in January, straining relations with economies of allies and rivals and fueling fears of higher inflation.

This latest move leaves Indian exporters facing among the highest US duties Trump has slapped on goods from overseas. Brazil is also grappling with 50% tariffs on its exports to the US."""

In [73]:
results = search_db(vector_store_id, "Which two articles would be most relevant to link to from an article\
    containing this snippet: \n{:s}".format(snippet))

In [74]:
for r in results:
    print(r.score)
    print(r.filename)
    print(r.content)
    print()

0.8160826278823857
https://www.theguardian.com/business/2022/jun/26/concerns-india-back-door-into-europe-for-russian-oil.txt
[Content(text='The stigma attached to the trade means some international companies involved in fuel supplies may attempt to mask its origins. Some energy firms rushed to cut shipments from Russia but industry watchers said some drivers in the south-east of England were still likely to still be filling up with diesel refined in Russia. State processors of oil are attempting to secure six-month supply contacts for Russian crude to India, Bloomberg reported this month. The trio of state refiners – Indian Oil Corp, Hindustan Petroleum and Bharat Petroleum – declined to answer questions on whether they were importing Russian oil or exporting it to Europe. Industry sources said tracking shipments of Russian oil to Europe via India is proving very difficult. “You’ll find that several shipments of crude will arrive at a port from different countries and be blended togeth